In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [ ]:
words=open("names.txt","r").read().splitlines()
chars=['.']+sorted(list(set("".join(words))))
stoi={s:i for i,s in enumerate(chars)}
itos={i:s for s,i in stoi.items()}

In [ ]:
def build_dataset(words,block_size):
    X,Y=[],[]
    for w in words:
        chs=w+'.'
        context=[0]*block_size
        for ch in chs:
            ix=stoi[ch]
            X.append(context)
            Y.append(ix)
            context=context[1:]+[ix]
    X=torch.tensor(X)
    Y=torch.tensor(Y)
    return X,Y

import random
random.seed(42)
random.shuffle(words)
n1=int(0.8*len(words))
n2=int(0.9*len(words))
Xtr,Ytr=build_dataset(words[:n1],3)
Xdev,Ydev=build_dataset(words[n1:n2],3)
Xte,Yte=build_dataset(words[n2],3)


In [ ]:
g=torch.Generator().manual_seed(2147483647)
C=torch.randn(27,10,generator=g)
W1=torch.randn(30,200,generator=g)
b1=torch.randn(200,generator=g)
W2=torch.randn(200,27,generator=g)
b2=torch.randn(27,generator=g)
para=[C,W1,b1,W2,b2]
for p in para:
    p.requires_grad=True
sum([p.nelement() for p in para])
batch_size=32

In [ ]:
index,lossi=[],[]
for i in range(20000):
    ix=torch.randint(0,Xtr.shape[0],(batch_size,))
    emb=C[Xtr[ix]]
    h=torch.tanh(emb.view(batch_size,-1)@W1+b1)
    logits=h@W2+b2
    loss=F.cross_entropy(logits,Ytr[ix])
    index.append(i)
    lossi.append(loss.item())
    lr=0.1 if i<10000 else 0.01
    for p in para:
        p.grad=None
    loss.backward()
    for p in para:
        p.data-=lr*p.grad

In [ ]:
plt.plot(index,lossi)
print(lossi[-1])

In [ ]:
g=torch.Generator().manual_seed(2147483647+10)
print(batch_size)
for _ in range(20):
    out=[]
    context=[0]*3
    while True:
        emb=C[torch.tensor([context])]
        h=torch.tanh(emb.view(1,-1)@W1+b1)
        logits=h@W2+b2
        probs=F.softmax(logits,dim=1)
        ix=torch.multinomial(probs,num_samples=1,generator=g).item()
        context=context[1:]+[ix]
        out.append(ix)
        if ix ==0 :
            break
    print("".join(itos[i] for i in out))